

__Set your own route planner for a given OSM map and GTFS file and query server on localhost for a detailed routes__


### OTP walkthrough

from: http://docs.opentripplanner.org/en/latest/Basic-Tutorial/

1. Install Java SDK
2. Download [latest](https://repo1.maven.org/maven2/org/opentripplanner/otp/1.4.0/otp-1.4.0-shaded.jar) .jar from [Maven](https://repo1.maven.org/maven2/org/opentripplanner/otp/1.4.0/)
3. Download gtfs.zip of a given city into a `folder` from `transit.land`
4. Download osm.pgf of your area from [here](https://www.interline.io/osm/extracts/) (need to login to get API token)
5. call from terminal `java -Xmx6G -jar otp-1.4.0-shaded.jar --build data_path --inMemory` changing `6G` as memory allocated (6GB) and `data_path` to your path with osm and gtfs files
6. if succesful you shall be able to open a web search in your browser: `http://localhost:8080/`
7. and query it via API, for instance: 'http://localhost:8080/otp/routers/default/plan?fromPlace=52.25177%2C20.92758&toPlace=52.20550%2C21.00517&time=1%3A02pm&date=04-09-2020&mode=TRANSIT%2CWALK&maxWalkDistance=5000&arriveBy=false'


In [1]:
import pandas as pd
import requests
import osmnx as ox

local imports

In [2]:
from main import make_query, parse_OTP_response
from utils import plot

set up

In [3]:
PATH = 'georequests_O(area10)_D.csv'
OTP_API = "http://localhost:8080/otp/routers/default/plan"

In [4]:
df = pd.read_csv(PATH)
G = ox.load_graphml('Krakow.graphml')
df1 = df.filter(['x','y','desti_x', 'desti_y', 'treq'], axis = 1).rename(
    columns = {'x': 'origin_x', 'y': 'origin_y', 'desti_x': 'destination_x', 'desti_y': 'destination_y'})
df1['treq'] = pd.to_datetime(df1.treq)

input

sample query (before you can query the server you need to run it -see run OTP server.ipynb in this folder)

[query synthax](http://dev.opentripplanner.org/apidoc/1.0.0/resource_PlannerResource.html)  
[response json structure](http://dev.opentripplanner.org/apidoc/1.0.0/json_Response.html)

In [5]:
df2 = df1.apply(lambda row: 
                 parse_OTP_response(requests.get(OTP_API, params=make_query(row.squeeze())).json()), axis=1)
df2[0]

{'success': True,
 'n_itineraries': 3,
 'duration': 2458,
 'walkDistance': 1014.8020375854327,
 'transfers': 1,
 'transitTime': 1560,
 'waitingTime': 81,
 'modes': [['WALK', 478, 608],
  ['BUS', 240, 944],
  ['WALK', 101, 130],
  ['TRAM', 1320, 6352],
  ['WALK', 238, 274]]}

In [6]:
for name in list(df2[0].keys()):
    vals = []
    for i in range(len(df2)):
        if df2[i]['success'] == True:
            vals.append(df2[i][name])
        else:
            vals.append(None)
    df1[name] = vals
df1[:3]

,origin_x,origin_y,destination_x,destination_y,treq,success,n_itineraries,duration,walkDistance,transfers,transitTime,waitingTime,modes
0,19.871552,50.018840,19.954225,50.045635,2024-03-28 07:54:33,True,3.0,2458.0,1014.802038,1.0,1560.0,81.0,"[[WALK, 478, 608], [BUS, 240, 944], [WALK, 101..."
1,19.873129,50.015181,19.934195,50.070585,2024-03-28 07:52:19,True,3.0,2987.0,831.435541,1.0,2040.0,261.0,"[[WALK, 238, 293], [BUS, 240, 944], [WALK, 101..."
2,19.885724,50.017353,19.896833,50.023390,2024-03-28 08:05:43,True,3.0,826.0,839.163955,0.0,120.0,2.0,"[[WALK, 348, 398], [TRAM, 120, 1182], [WALK, 3..."


In [7]:
# test
row = df1.sample(1).squeeze()
print(row)
print()
query = make_query(row)
print(query)
print()
r = requests.get(OTP_API, params=query)
route = parse_OTP_response(r.json())
print(route)

origin_x                                                 19.882898
origin_y                                                 50.019171
destination_x                                            19.906643
destination_y                                            50.030131
treq                                           2024-03-28 08:10:10
success                                                       True
n_itineraries                                                  3.0
duration                                                    1186.0
walkDistance                                           1169.584692
transfers                                                      0.0
transitTime                                                  180.0
waitingTime                                                    2.0
modes            [[WALK, 589, 690], [TRAM, 180, 1596], [WALK, 4...
Name: 59, dtype: object

{'fromPlace': '50.01917132,19.88289838', 'toPlace': '50.03013095170665,19.90664300481499', 'time': '08:1

#### visualization

In [8]:
plot(r.json()['plan'], color = 'green')

Trip from (19.8829,50.0192) to (19.9066,50.0301) at 1711609800000. 
3 connections found. 
Best one is 20min (1170m walk, 0 transfer(s), wait time 0.03min)

LEG 	 MODE 	DIST 	TIME
-----------------------------
1	WALK	691	589
2	TRAM	1596	180
3	WALK	478	415


output

In [10]:
# df1.to_csv("georequests_O(area10)_D_PT.csv")

full response

In [11]:
# r.json()['requestParameters']

---